## Outlier removal

In [ ]:
def display_inlier_outlier(cloud, ind, mytitle = None, mytuples = None):
    inlier_cloud = cloud.select_by_index(ind) #https://github.com/intel-isl/Open3D/issues/1860 
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    if mytitle is None:
        mytitle = "Showing outliers (red) and inliers (gray) "
    
    if mytuples is not None:
        for i in range(len(mytuples)):
            mytitle = mytitle+str(mytuples[i][0])+"="+str(mytuples[i][1])+"-"
        mytitle = mytitle[:-1]

    outlier_cloud.paint_uniform_color([1, 0, 0]) # print outliers in red
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8]) # print inliers in grey
    
    # use draw geometries to plot
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud],
                                       width=1000, height=800,
                                       window_name='%s'%(mytitle))

## View with custom Visualizer 
http://www.open3d.org/docs/release/tutorial/visualization/customized_visualization.html

In [11]:
import time
def custom_draw_geometry_outliers(cloud, ind, 
                                  mytitle = None, mytuples = None, 
                                  params = None, #camera parameters,json file (P)
                                  fov_step  = None, 
                                  configuration_file = None, #object properties ,json file (O)
                                  rotate = False):
    """
    cloud = point cloud
    ind = index of outliers to be marked in red 
    mytitle = general title
    mytuples = zip(featurelabels,feature) if you want to print which parameter are being used
    fov_step = change of field of View
    
    """
    
    #1- initialize the visualizer
    vis = o3d.visualization.Visualizer()
       
   
    #2- set up naming system
    if mytitle is None:
        mytitle = "Showing outliers (red) and inliers (gray) "
    
    if mytuples is not None:
        for i in range(len(mytuples)):
            mytitle = mytitle+"-"+str(mytuples[i][0])+"="+str(mytuples[i][1])+"-"
        mytitle = mytitle[:-1]

    
    #- create window
    vis.create_window(window_name=mytitle, 
                      width=1000, height=800, 
                      left=50, top=50, 
                      visible=True)
    
    #- get inner and outer cloud
    inlier_cloud = cloud.select_by_index(ind) #https://github.com/intel-isl/Open3D/issues/1860 
    outlier_cloud = cloud.select_by_index(ind, invert=True)
    outlier_cloud.paint_uniform_color([1, 0, 0]) # print outliers in red
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8]) # print inliers in grey
    
    # - add geometries
    vis.add_geometry(inlier_cloud)
    vis.add_geometry(outlier_cloud)
    
    
    # - get view control 
    ctr = vis.get_view_control()
    
    # - set view parameters if any  
    if params is not None: 
        # params = "./ScreenCamera_2021-01-10-15-00-31.json" eg, obtainable by pressing P while visualizing the pcl
        print ("loading parameters: \n ",params)
        parameters = o3d.io.read_pinhole_camera_parameters(params)
        ctr.convert_from_pinhole_camera_parameters(parameters)
    
    if fov_step is not None: 
        #ctr = vis.get_view_control()
        print("Field of view (before changing) %.2f" % ctr.get_field_of_view())
        ctr.change_field_of_view(step=fov_step/5)
        print("Field of view (after changing) %.2f" % ctr.get_field_of_view())
    
    if configuration_file is not None:
        print ("loading configuration file: \n ",configuration_file)
        vis.get_render_option().load_from_json(str(configuration_file)) 
        
    #define pcd
    pcd = inlier_cloud + outlier_cloud
        
    if rotate == True:
             
        def rotate_view(vis):
            ctr = vis.get_view_control()
            ctr.rotate(5.0, 0.0) #(speed, direction? )
            return False
        
        o3d.visualization.draw_geometries_with_animation_callback([pcd],
                                                           rotate_view,
                                                           window_name=mytitle, 
                                                           width=1000, height=800, 
                                                           left=50, top=50, 
                                                          )
    else: 
        vis.run()
    
    #time.sleep(1)
    #save the image as png
    #vis.capture_screen_image(mytitle+'.png')

    #vis.remove_geometry(pcd)
    #close
    vis.destroy_window()

In [14]:
# build on this to get your viewer
def custom_draw_geometry(pcd, 
                         mytitle = None, mytuples = None,
                         params =None, 
                         configuration_file = None, 
                         take_screen_shot = False,
                         rotate = False):
    # The following code achieves the same effect as:
    # o3d.visualization.draw_geometries([pcd])
    
    if isinstance(pcd,list):
        
        #count of point clouds
        l = len(pcd)
        
        if isinstance(mytitle,list):
            mmytitle = mytitle
        else:
            mmytitle = ["custom draw geometry of pcl_"+str(i+1) for i in list(range(l))]
            
        if isinstance(mytuples,list):
            mmytuples = mytuples
        else:
            mmytuples = [None]*l
        
        
        for i in list(range(len(pcd))):
            custom_draw_geometry(pcd[i], 
                             mytitle = mmytitle[i], mytuples = mmytuples[i],
                             params =params, 
                             configuration_file =configuration_file, 
                             take_screen_shot = take_screen_shot,
                             rotate = rotate)
    
    
    else: 
        # 1- initialize the visualizer
        vis = o3d.visualization.Visualizer()

        #2- set up naming system
        if mytitle is None:
            mytitle = "custom_draw_geometry "

        if mytuples is not None:
            for i in range(len(mytuples)):
                mytitle = mytitle+"-"+str(mytuples[i][0])+"="+str(mytuples[i][1])+"-"
            mytitle = mytitle[:-1]

        # 3- create widow, otherwise the kernel crashes
        vis.create_window(window_name=mytitle, 
                  width=1000, height=800, 
                  left=50, top=50, 
                  visible=True)

        # 4- add the geometry before taking view control
        #pcd.paint_uniform_color([0.3, 0.3, 0]) 
        vis.add_geometry(pcd)

        # 5- take view control after having added the geometry and before vis.run
        ctr = vis.get_view_control()
        if params is not None: 
            print ("loading parameters: \n ",params)
            parameters = o3d.io.read_pinhole_camera_parameters(params)
            ctr.convert_from_pinhole_camera_parameters(parameters)

        if configuration_file is not None:
            print ("loading configuration file: \n ",configuration_file)
            vis.get_render_option().load_from_json(str(configuration_file)) 

        if rotate == True:      
            def rotate_view(vis):
                ctr = vis.get_view_control()
                ctr.rotate(5.0, 0.0) #(speed, direction? )
                return False

            o3d.visualization.draw_geometries_with_animation_callback([pcd],
                                                               rotate_view,
                                                               window_name=mytitle, 
                                                               width=1000, height=800, 
                                                               left=50, top=50, 
                                                              )
        else: 
            vis.run()




        #vis.run()

        if take_screen_shot == True:
            vis.capture_screen_image(mytitle+'.png')

        vis.destroy_window()

In [13]:
def custom_draw_geometry_load_option(pcd, configuration_file):
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window(window_name='custom_load_json', 
                      width=1000, height=800, 
                      left=50, top=50)
    vis.add_geometry(pcd)
    vis.get_render_option().load_from_json(str(configuration_file))    
    vis.run()
    #vis.destroy_window()

In [9]:
def custom_draw_geometry_with_custom_fov(pcd, fov_step):
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name='custom_draw_geometry', 
                      width=1000, height=800, 
                      left=50, top=50, 
                      visible=True)
    
    vis.add_geometry(pcd)
    ctr = vis.get_view_control()
    print("Field of view (before changing) %.2f" % ctr.get_field_of_view())
    ctr.change_field_of_view(step=fov_step)
    print("Field of view (after changing) %.2f" % ctr.get_field_of_view())
    vis.run()
    vis.destroy_window()

In [ ]:
def custom_draw_geometry_with_rotation(pcd):

    def rotate_view(vis):
        ctr = vis.get_view_control()
        ctr.rotate(10.0, 0.0)
        return False

    o3d.visualization.draw_geometries_with_animation_callback([pcd],
                                                              rotate_view)

In [ ]:

def visualize_tooth(t):
    geometry_list = [external_ply[t].paint_uniform_color([0.5, 0, 0]),
                     internal_ply[t].paint_uniform_color([0, 0.5, 0]),
                     upper_ply[t].paint_uniform_color([0, 0, 0.5])
                    ]
    o3d.visualization.draw_geometries(geometry_list,
                                      width=1000, height=800,
                                      window_name='3 raw views of tooth %s'%t
                                     )

def draw_registration_result(source, target, transformation, title = ""):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      width=1000, height=800,
                                      window_name='Open3D-'+str(title)
                                     )